In [8]:
from db import db
import pymongo
import time

one = next(db.repka_logs.aggregate([
    {'$sort': { 'time': -1 }},
    { '$match': { 'is_viewed': { "$ne": True }, 'app': { "$eq": "hagi" } } },
    {'$skip': 2 },
    {'$limit': 1 },
]))


text = []

for one in db.repka_logs.find({ "session_id": one['session_id'] }).sort([("message_id", 1)]):
    text.append(f"> {one['request']}\n< {one['response']}\n")

print("\n".join(text))



db.repka_logs.update_one({'session_id': one['session_id']}, {"$set": { 'is_viewed': True }} )

> 
< Хаги - кукла из игры.
 Ожила так что беги.
 Я ХАГИ ВАГИ. Я буду 

> а а
< а а

> б б
< б б

> але
< але

> вова
< вова

> дима
< дима

> димос
< димос

> а что в игре правила какие
< а что в игре правила какие

> какие правила в игре
< какие правила в игре

> ну у тебя какие ты же эту игру сделал
< ну у тебя какие ты же эту игру сделал

> ты эту игру сделал
< ты эту игру сделал

> ты
< ты

> ты
< ты

> ты
< ты

> ты
< ты

> ты
< ты

> ты
< ты

> это ты сделал
< это ты сделал

> это ты игру сделал
< это ты игру сделал

> ты эту игру сделал
< ты эту игру сделал

> я тебе говорю это ты игру
< я тебе говорю это ты игру

> это ты сделала игру так что давай какие правила рассказывай быстрей ты у меня
< это ты сделала игру так что давай какие правила рассказывай быст

> ты совсем ку ку
< ты совсем ку ку

> какой ты просто дебил
< какой ты просто дебил



In [2]:
# db.command({ 'collStats' : "repka_logs", 'scale': 1024*1024 })

In [1]:
from db import db

for one in db.repka_logs.find({ 'is_viewed': { "$ne": True }, 'app': { "$eq": "hagi" } }).sort([("message_id", 1)]):
    if one['request']: print(f"> {one['request']}")

> включи алиса игры про хаги ваги
> найти игру
> игра
> одевай гис давай поговорим с тобой
> игра
> игра
> объясни правила
> алиса включи новенькие записки
> зачем мне закрывать глаза
> алиса давай поиграем в повторюшка
> хаги ваги отвали
> страшный сон
> данил конченный
> эй хаги ваги
> эй ты
> ох а где ваги
> ты дебил ты дебил ты дебил
> давай поиграем хаги ваги
> алиса включи игру крокодил
> бука бука
> алеша попович дурак
> повтори все за мной то что я говорю
> окей
> как дела привет
> хочу хаги ваги начать игру за братьев
> а ты что мне хочешь сказать хаги ваги
> обнаружен танк
> никто не звонил
> хаги ваги бе бе бе
> ля ля ля ля ля ля убей мамона
> привет хаги ваги
> хаги ваги ты что человек
> повтори
> как так
> ничего
> запусти навык енот повторюшка
> муха
> что нибудь
> дай мне киси миси
> зачем мне бежать
> я буду с тобой играть хочу
> ну привет
> ну ты папа у меня только найди вообще не найдешь никогда
> принеси мне только хорошо что я с тобой играть не хочу
> слышишь ты теб